In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input/"))

from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))


big-mart-sales-dataset
bigmartsale



In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#%%capture
!pip install fastai==0.7.0

    100% |████████████████████████████████| 122kB 5.8MB/s 
    100% |████████████████████████████████| 496.4MB 61kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
  Found existing installation: fastai 1.0.51
    Uninstalling fastai-1.0.51:
      Successfully uninstalled fastai-1.0.51


In [4]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [5]:
PATH = "../input/bigmartsale"

In [6]:
!ls  {PATH}

SampleSubmission.csv  Test.csv	Train.csv


In [7]:
df_raw = pd.read_csv(f'{PATH}/Train.csv', low_memory=False, 
                    )
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [8]:
df_test = pd.read_csv(f'{PATH}/Test.csv', low_memory=False, 
                     )
df_test.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [9]:
df_raw.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [10]:
sample_sub = pd.read_csv(f'{PATH}/SampleSubmission.csv', low_memory=False)
sample_sub.columns

Index(['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'], dtype='object')

In [11]:
display_all(df_raw.tail().T)

,8518,8519,8520,8521,8522
Item_Identifier,FDF22,FDS36,NCJ29,FDN46,DRG01
Item_Weight,6.865,8.38,10.6,7.21,14.8
Item_Fat_Content,Low Fat,Regular,Low Fat,Regular,Low Fat
Item_Visibility,0.0567834,0.0469824,0.0351863,0.145221,0.0448783
Item_Type,Snack Foods,Baking Goods,Health and Hygiene,Snack Foods,Soft Drinks
Item_MRP,214.522,108.157,85.1224,103.133,75.467
Outlet_Identifier,OUT013,OUT045,OUT035,OUT018,OUT046
Outlet_Establishment_Year,1987,2002,2004,2009,1997
Outlet_Size,High,NaN,Small,Medium,Small
Outlet_Location_Type,Tier 3,Tier 2,Tier 2,Tier 3,Tier 1


 In this case Kaggle tells us what metric to use: RMSLE (root mean squared log error) between the actual and predicted auction prices. Therefore we take the log of the prices, so that RMSE will give us what we need.

In [12]:
#print(df_raw.dtypes)
#print(df_raw.shape)

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call train_cats to convert strings to pandas categories.Pandas categories store a numeric code for each category at the backend.

In [13]:
df_test['Item_Fat_Content'].unique()

array(['Low Fat', 'reg', 'Regular', 'LF', 'low fat'], dtype=object)

In [14]:
for i in range(len(df_raw)):
    if (df_raw['Item_Fat_Content'][i] == 'low fat'):
        df_raw['Item_Fat_Content'][i]='Low Fat'
    elif df_raw['Item_Fat_Content'][i] == 'LF':
        df_raw['Item_Fat_Content'][i]='Low Fat'
    elif df_raw['Item_Fat_Content'][i] == 'reg':
        df_raw['Item_Fat_Content'][i]= 'Regular'
    else:
        continue

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [15]:
for i in range(len(df_test)):
    if (df_test['Item_Fat_Content'][i] == 'low fat'):
        df_test['Item_Fat_Content'][i]='Low Fat'
    elif df_test['Item_Fat_Content'][i] == 'LF':
        df_test['Item_Fat_Content'][i]='Low Fat'
    elif df_test['Item_Fat_Content'][i] == 'reg':
        df_test['Item_Fat_Content'][i]= 'Regular'
    else:
        continue

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
train_cats(df_raw)
apply_cats(df_test,df_raw)

Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.
Note that only one variable is changed and remaining are not.

We're still not quite done - for instance we have lots of missing values, which we can't pass directly to a random forest.

Calculating the % of M.V

In [17]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Item_Fat_Content             0.000000
Item_Identifier              0.000000
Item_MRP                     0.000000
Item_Outlet_Sales            0.000000
Item_Type                    0.000000
Item_Visibility              0.000000
Item_Weight                  0.171653
Outlet_Establishment_Year    0.000000
Outlet_Identifier            0.000000
Outlet_Location_Type         0.000000
Outlet_Size                  0.282764
Outlet_Type                  0.000000
dtype: float64

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [18]:
df, y, nas = proc_df(df_raw, 'Item_Outlet_Sales')
dft, y_t,nas_t = proc_df(df_test, na_dict = nas)

In [19]:
from sklearn.model_selection import train_test_split
def split_vals(df,y):
    X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.2)
    return X_train, X_valid, y_train, y_valid


In [20]:
X_train, X_valid, y_train, y_valid = split_vals(df,y)

In [21]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    """
    Return: rmse of training vs actual
            rmse on validation vs actual
            r2 of training
            r2 on validation
    """
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

> Speeding things up -
It turns out that one of the easiest ways to avoid over-fitting is also one of the best ways to speed up analysis: subsampling. Let's return to using our full dataset, so that we can demonstrate the impact of this technique.

The basic idea is this: rather than limit the total amount of data that our model can access, let's instead limit it to a different random subset per tree. That way, given enough trees, the model can still see all the data, but for each individual tree it'll be just as fast as if we had cut down to 20000

Bagging

Out-of-bag (OOB) score
Is our validation set worse than our training set because we're over-fitting, or because the validation set is for a different time period, or a bit of both? With the existing information we've shown, we can't tell. However, random forests have a very clever trick called out-of-bag (OOB) error which can handle this (and more!)

The idea is to calculate error on the training set, but only include the trees in the calculation of a row's error where that row was not included in training that tree. This allows us to see whether the model is over-fitting, without needing a separate validation set.

This also has the benefit of allowing us to see whether our model generalizes, even if we only have a small amount of data so want to avoid separating some out to create a validation set.

This is as simple as adding one more parameter to our model constructor. We print the OOB error last in our print_score function below.

In [22]:
#plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(30)]);


Another way to reduce over-fitting is to grow our trees less deeply. We do this by specifying (with min_samples_leaf) that we require some minimum number of rows in every leaf node. This has two benefits:

There are less decision rules for each leaf node; simpler models should generalize better
The predictions are made by averaging more rows in the leaf node, resulting in less volatility

In [23]:
"""m = RandomForestRegressor(n_estimators=22, min_samples_leaf=3, max_features='sqrt', n_jobs=-1, oob_score=True)
m.fit(df, y)
print(m.oob_score_)#submission2
m = RandomForestRegressor(n_estimators=22, min_samples_leaf=5, max_features='sqrt', n_jobs=-1, oob_score=True)
m.fit(df, y)
print(m.oob_score_)#submission3
m = RandomForestRegressor(n_estimators=22, min_samples_leaf=5, max_features='log2', n_jobs=-1, oob_score=True)
m.fit(df, y)
print(m.oob_score_)"""#submission4
"""m = RandomForestRegressor(n_estimators=80, min_samples_leaf=10, max_features='sqrt', n_jobs=-1, oob_score=True)
m.fit(df, y)"""#submission5 with grid search params score is 1157 rank 945
m = RandomForestRegressor(n_estimators=90, min_samples_leaf=11, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(df, y)#submission 6

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.5, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=11, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [24]:
predictions = m.predict(dft)
submission = pd.DataFrame({'Item_Identifier': df_test.Item_Identifier, 'Outlet_Identifier' : df_test.Outlet_Identifier,
                           'Item_Outlet_Sales': predictions})
submission.to_csv('submission6.csv', index=False)

**Feature importance**

In [25]:
#k-fold cross validation
#for small dataset its better to use high value of K
"""from sklearn.model_selection import RepeatedKFold, cross_val_score
from statistics import mean 
kf = RepeatedKFold(n_splits=20, n_repeats=1, random_state=42) 
clf = RandomForestRegressor(n_estimators=22, min_samples_leaf=5, max_features='sqrt', n_jobs=-1)
lis = cross_val_score(clf, df, y, cv=kf, n_jobs=-1, scoring='r2')
print("mean validation r2 is: "+ str(lis.mean()))
print("the standard deviation is {} this mean how far is the each lis elements from lis.mean()".format(lis.std()))
"""

'from sklearn.model_selection import RepeatedKFold, cross_val_score\nfrom statistics import mean \nkf = RepeatedKFold(n_splits=20, n_repeats=1, random_state=42) \nclf = RandomForestRegressor(n_estimators=22, min_samples_leaf=5, max_features=\'sqrt\', n_jobs=-1)\nlis = cross_val_score(clf, df, y, cv=kf, n_jobs=-1, scoring=\'r2\')\nprint("mean validation r2 is: "+ str(lis.mean()))\nprint("the standard deviation is {} this mean how far is the each lis elements from lis.mean()".format(lis.std()))\n'

In [26]:
#Before Grid Search we need to fit the model once
#m = RandomForestRegressor(n_estimators=22, min_samples_leaf=5, max_features='sqrt', n_jobs=-1, oob_score=True)
#m.fit(df, y)

In [27]:
#Fitting the model after kFold cross validation by using Grid Search
"""from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [40, 70,80, 90, 100], 'min_samples_leaf': [7, 8, 9, 10, 11], 'max_features': ['sqrt', 'log2', 0.5]},

             ]
grid_search = GridSearchCV(estimator = m,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(df, y)
best_accuracy = grid_search.best_score_#given by mean of 10 folds
best_parameters = grid_search.best_params_
print ("best accuracy is {}".format(best_accuracy))
print ("best parametrs are {}".format(best_parameters))"""


'from sklearn.model_selection import GridSearchCV\nparameters = [{\'n_estimators\': [40, 70,80, 90, 100], \'min_samples_leaf\': [7, 8, 9, 10, 11], \'max_features\': [\'sqrt\', \'log2\', 0.5]},\n\n             ]\ngrid_search = GridSearchCV(estimator = m,\n                           param_grid = parameters,\n                           scoring = \'r2\',\n                           cv = 10,\n                           n_jobs = -1)\ngrid_search = grid_search.fit(df, y)\nbest_accuracy = grid_search.best_score_#given by mean of 10 folds\nbest_parameters = grid_search.best_params_\nprint ("best accuracy is {}".format(best_accuracy))\nprint ("best parametrs are {}".format(best_parameters))'

In [28]:
#sorted(sklearn.metrics.SCORERS.keys())

In [29]:
#fi = rf_feat_importance(m, df); fi[:]

XGboost 

In [30]:
#from sklearn.metrics import accuracy_score
import xgboost
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [31]:
"""xgb = xgboost.XGBRegressor(n_estimators=80, learning_rate=0.09, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
print_score(xgb)"""

'xgb = xgboost.XGBRegressor(n_estimators=80, learning_rate=0.09, gamma=0, subsample=0.75,\n                           colsample_bytree=1, max_depth=7)\nxgb.fit(X_train,y_train)\nprint_score(xgb)'

In [32]:
#Fitting the model after kFold cross validation by using Grid Search
"""from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [10, 20, 40, 80], 'min_samples_leaf': [3, 5, 7, 10], 'max_features': ['sqrt', 'log2', 0.5]},

             ]
grid_search = GridSearchCV(estimator = m,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(df, y)
best_accuracy = grid_search.best_score_#given by mean of 10 folds
best_parameters = grid_search.best_params_"""

"from sklearn.model_selection import GridSearchCV\nparameters = [{'n_estimators': [10, 20, 40, 80], 'min_samples_leaf': [3, 5, 7, 10], 'max_features': ['sqrt', 'log2', 0.5]},\n\n             ]\ngrid_search = GridSearchCV(estimator = m,\n                           param_grid = parameters,\n                           scoring = 'r2',\n                           cv = 10,\n                           n_jobs = -1)\ngrid_search = grid_search.fit(df, y)\nbest_accuracy = grid_search.best_score_#given by mean of 10 folds\nbest_parameters = grid_search.best_params_"